In [1]:
!pip install -q tensorflow-hub
!pip install -q tfds-nightly

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 458.3 MB 18 kB/s s eta 0:00:01     |█                               | 14.4 MB 1.6 MB/s eta 0:04:29     |██                              | 27.4 MB 1.6 MB/s eta 0:04:22     |███▍                            | 48.4 MB 44.6 MB/s eta 0:00:10     |███████                         | 98.9 MB 49.5 MB/s eta 0:00:08     |█████████▍                      | 134.9 MB 49.5 MB/s eta 0:00:07     |█████████████████▏              | 245.1 MB 60.0 MB/s eta 0:00:04     |█████████████████████████▌      | 365.4 MB 49.9 MB/s eta 0:00:02
     |████████████████████████████████| 65 kB 5.6 MB/s  eta 0:00:01
     |████████████████████████████████| 462 kB 49.0 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 7.1 MB/s  eta 0:00:01
     |████████████████████████████████| 4.3 MB 11.5 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 1.6 MB/s  eta 0:00:01
     |███████████████████

In [4]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

print('버전 : ', tf.__version__)
print('즉시 실행 모드 : ', tf.executing_eagerly())
print('허브 버전 : ', hub.__version__)
print('GPU', '사용가능' if tf.config.experimental.list_physical_devices('GPU') else '사용 불가능')

버전 :  2.6.0
즉시 실행 모드 :  True
허브 버전 :  0.12.0
GPU 사용가능


In [5]:
import pandas as pd
df = pd.read_csv('prec.csv')
df.head()

,title,text,date_d,date_i,change
0,지난달 개미 9조 샀지만…수익은 지지부진,개인투자자가 지난달 반도체와 전기차 관련주에 집중 투자했지만 손실을 봤던 것으로 ...,2021-08-01,2021-08-02,1
1,"""재테크하려면 지금 당장 가입하라""…증권사 전화 불나게 만든 상품은?",[사진 = 이승환 기자] 다양한 금융상품에 투자할 수 있고 세제 혜택도 있어 '만능...,2021-08-01,2021-08-02,1
2,"""가전제품이 20초에 한대씩 팔리다니""…50년의 기다림 끝에 대세됐다",LG전자가 1968년 국내 가전업계에서 처음 선보인 창문형 에어컨 `GA-111`....,2021-08-01,2021-08-02,1
3,"[단독] 삼성 현대차 SK 또 발등에 불…""제2사드보복 될라""",◆ 美 베이징올림픽 때리기 ◆미 백악관 [사진 출처 = 연합뉴스] 미국 의회와 인권...,2021-08-01,2021-08-02,1
4,'7만전자' 늪 빠진 '최고실적' 삼성전자…의미있는 점프 하려면,[출처 : 매일경제DB] 2분기 실적도 좋았고 3분기 전망도 좋은 삼성전자의 주가가...,2021-08-01,2021-08-02,1


In [7]:
text_d=df[['text','change']]

In [21]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(text_d, test_size=0.2, random_state=0)

In [15]:
train_data, validation_data, test_data = tfds.load(
    name='imdb_reviews',
    split=['train[:60%]', 'train[60%:]', 'test'],
    as_supervised=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling imdb_reviews-train.tfrecord...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling imdb_reviews-test.tfrecord...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling imdb_reviews-unsupervised.tfrecord...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /home/lab21/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [10]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

NameError: name 'Series' is not defined

In [ ]:
embedding = 'https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1'
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
with tf.device('/GPU:0'):
    history = model.fit(train_data.shuffle(10000).batch(512),
                        epochs=20,
                        validation_data=validation_data.batch(512),
                        verbose=1)

In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
    print('%s : %.3f' % (name, value))